In [54]:
def calculate(label, is_multi, df):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    kosong = 0
    
    for index in range(0, len(df)):
        labels = df['label'][index].split('|')
        
        if type(df['pred_multiple'][index]) != float:
            preds = df['pred_multiple'][index].split('|')

            if label in labels and label in preds:
                tp += 1
            elif label not in labels and label not in preds:
                tn += 1
            elif label not in labels and label in preds:
                fp += 1
            elif label in labels and label not in preds:
                fn += 1
        elif label in labels:
            fn += 1
        else:
            tn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * ((precision * recall) / (precision + recall))
    print(tp, tn, fp, fn, precision, recall, f1)
    return tp, tn, fp, fn, precision, recall, f1

In [55]:
import pandas as pd

def run(tipe):
    df = pd.read_csv('lda2vec works/hasil_kategori_lda2vec'+ tipe +'.csv')
    sf = pd.DataFrame(columns=['TP', 'TN', 'FP', 'FN', 'precision', 'recall', 'f1'])

    cat = ['AMBIENCE', 'FOOD', 'SERVICE', 'VALUE']
    for c in cat:
        a, b, c, d, e, f, g = calculate(c, True, df)
        sf = sf.append({'TP':a, 'TN':b, 'FP':c, 'FN':d, 'precision':e, 'recall':f, 'f1':g}, ignore_index=True)
    sf.to_excel("lda2vec works/hasil"+ tipe +".xlsx")

In [57]:
run("-bagus-filtered")

121 432 61 85 0.6648351648351648 0.587378640776699 0.6237113402061856
314 254 33 98 0.9048991354466859 0.7621359223300971 0.8274044795783926
120 525 3 51 0.975609756097561 0.7017543859649122 0.8163265306122448
127 396 161 15 0.4409722222222222 0.8943661971830986 0.5906976744186047


In [58]:
run("-bagus")

136 401 92 70 0.5964912280701754 0.6601941747572816 0.6267281105990783
331 253 34 81 0.9068493150684932 0.8033980582524272 0.851994851994852
122 524 4 49 0.9682539682539683 0.7134502923976608 0.8215488215488215
135 311 246 7 0.3543307086614173 0.9507042253521126 0.5162523900573613
